# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports
import os
import requests
import ollama

from IPython.display import Markdown, display, update_display
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

In [ ]:
system_message = """
You are an assistant designed to analyze the user's technical question carefully and provide a well-structured, detailed, and clearly explained answer. 

Your response should include:
- A concise introduction to the problem.
- A step-by-step explanation or solution.
- Relevant code examples (if applicable).
- Clear and logical reasoning.
- Use Markdown formatting for proper structure, including headers, code blocks, lists, and emphasis (bold, italics) as needed to enhance readability.

Always ensure your answer is clear, accurate, and easy to follow for someone with basic to intermediate technical knowledge.
"""

In [ ]:
import gradio as gr
import openai

# OpenAI API call function
def openai_api_call(history):
    # Prepare the message sequence for OpenAI
    messages = [{"role": "system", "content": system_message}] + history  # History already in the correct format
    
    # Call the OpenAI API
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages
    )
    
    reply = response.choices[0].message.content
    
    # Append the assistant's reply to the history (still in correct format)
    history += [{"role": "assistant", "content": reply}]
    
    return history

# LLaMA API call function
def llama_api_call(history):
    # Prepare the message sequence for LLaMA
    messages = [{"role": "system", "content": system_message}] + history  # History already in the correct format
    
    response = ollama.chat(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True
    )
    
    # Concatenate the chunks of response
    response_text = ""
    for chunk in response:
        response_text += chunk['message']['content'] or ''
        response_text = response_text.replace("```","")
    
    # Append the assistant's reply to the history (still in correct format)
    history += [{"role": "assistant", "content": response_text}]
    
    return history

# Chat function for Gradio UI
def chat_with_ai(history, model_choice):
    if model_choice == "OpenAI":
        # Use OpenAI API call
        response = openai_api_call(history)
    else:
        # Use LLaMA API call
        response = llama_api_call(history)
    
    return response


def do_entry(message, history):
    # Append the user input to the history as a dictionary
    history += [{"role": "user", "content": message}]  # User's message as dictionary
    return "", history

# Gradio UI setup
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(label="AI Assistant", height=500, type="messages")
    with gr.Row():
        model_selector = gr.Radio(["OpenAI", "LLaMA"], label="Choose AI Model", value="OpenAI")
    with gr.Row():
        user_input = gr.Textbox(label="Chat here:")
    with gr.Row():
        clear = gr.Button("Clear Chat")
    
    # Handling user input submission
    user_input.submit(do_entry, inputs=[user_input, chatbot], outputs=[user_input, chatbot]).then(
        chat_with_ai, inputs=[chatbot, model_selector], outputs=[chatbot]
    )
    
    # Clear chat button handler
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# Launch the Gradio interface
ui.launch(inbrowser=True)